In [1]:
# Initialize Everything First! Run this cell before anything else

# Import required libraries
import re
import json
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import io

# Chess library - install if needed
try:
    import chess
    import chess.pgn
except ImportError:
    print("Installing python-chess...")
    !pip install python-chess
    import chess
    import chess.pgn

# RealCommentaryGenerator class
class RealCommentaryGenerator:
    """Generates chess commentary based on actual Chess Network and Hikaru patterns"""
    
    def __init__(self):
        pass
    
    def generate_opening_commentary(self, metadata: Dict, moves: List[str]) -> str:
        """Generate opening commentary mixing both styles"""
        opening_moves = " ".join(moves[:6])  # First 6 moves
        
        # Chess Network style intro
        jerry_intro = f"Hi everyone, it's Jerry. I have an excellent game to share with you from {metadata.get('event', 'an online tournament')}. "
        jerry_intro += f"This is a {metadata.get('date', 'recent')} game between {metadata['white']} playing white against {metadata['black']}. "
        
        # Opening analysis
        jerry_analysis = f"We're looking at {opening_moves}. This demonstrates classical opening principles with both sides developing their pieces harmoniously. "
        
        # Hikaru style commentary
        hikaru_comment = f"Now, going into this position, both players are following standard theory, but {metadata['white']} could have also considered other setups. "
        hikaru_comment += f"Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. "
        
        return jerry_intro + jerry_analysis + hikaru_comment
    
    def generate_middlegame_commentary(self, move_data: Dict, critical_move: str = None) -> str:
        """Generate middlegame commentary for critical positions"""
        player = move_data.get('player', 'White')
        
        # Chess Network analysis
        jerry_analysis = f"Now let's examine this position carefully. {player} has just played {critical_move}, and this creates several interesting possibilities. "
        jerry_analysis += f"The key question here is how {player} should continue. If we look at the pawn structure, "
        jerry_analysis += f"we can see that {player} has good piece coordination and central control. "
        
        # Hikaru quick assessment  
        hikaru_comment = f"Chat, this position is actually pretty interesting. Like, {player} has some ideas here, "
        hikaru_comment += f"but the opponent definitely has counterplay. I mean, it's not completely winning, but there are chances. "
        
        # Combined tactical insight
        tactical_note = f"The computer evaluation here shows this is approximately equal, but in a practical game, "
        tactical_note += f"the player with better time management and tactical vision will likely prevail. "
        
        return jerry_analysis + hikaru_comment + tactical_note
    
    def generate_endgame_commentary(self, metadata: Dict, final_moves: List[str]) -> str:
        """Generate endgame commentary"""
        result = metadata.get('result', '1-0')
        winner = 'White' if result == '1-0' else 'Black' if result == '0-1' else 'both players'
        
        # Chess Network endgame principles
        jerry_endgame = f"In this endgame position, we can see the importance of precise technique. "
        jerry_endgame += f"The final moves {' '.join(final_moves[-3:])} demonstrate excellent endgame understanding. "
        jerry_endgame += f"This is exactly how you want to convert a winning position - methodically and accurately. "
        
        # Hikaru result assessment
        hikaru_result = f"And that's the game! {winner} gets the win with some really nice play. "
        hikaru_result += f"Chat, this was actually a pretty instructive game. Like, both players showed good opening preparation, "
        hikaru_result += f"but ultimately {winner} was just more accurate in the critical moments. "
        
        return jerry_endgame + hikaru_result
    
    def analyze_game_for_commentary(self, pgn_string: str) -> Dict:
        """Analyze entire game and generate mixed-style commentary"""
        try:
            # Parse the game
            metadata, moves = self.parse_pgn_basic(pgn_string)
            
            # Extract move strings
            move_strings = []
            for move in moves:
                if move.white_move:
                    move_strings.append(f"{move.move_number}.{move.white_move}")
                if move.black_move:
                    move_strings.append(f"{move.black_move}")
            
            commentary_sections = {
                'metadata': metadata,
                'opening_commentary': self.generate_opening_commentary(metadata, move_strings[:12]),
                'middlegame_commentary': self.generate_middlegame_commentary(
                    {'player': metadata['white']}, 
                    move_strings[12] if len(move_strings) > 12 else "the key move"
                ),
                'endgame_commentary': self.generate_endgame_commentary(metadata, move_strings),
                'full_game_summary': self.generate_game_summary(metadata, len(moves))
            }
            
            return commentary_sections
            
        except Exception as e:
            return {'error': f"Failed to generate commentary: {str(e)}"}
    
    def parse_pgn_basic(self, pgn_string: str) -> Tuple[Dict, List]:
        """Basic PGN parsing for commentary generation"""
        pgn_io = io.StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)
        
        if not game:
            raise ValueError("Invalid PGN format")
        
        metadata = {
            'white': game.headers.get('White', 'Unknown'),
            'black': game.headers.get('Black', 'Unknown'), 
            'result': game.headers.get('Result', '*'),
            'date': game.headers.get('Date', 'Unknown'),
            'event': game.headers.get('Event', 'Unknown')
        }
        
        # Simplified move parsing
        moves = []
        board = game.board()
        move_number = 1
        
        for move in game.mainline_moves():
            is_white_move = board.turn == chess.WHITE
            san_move = board.san(move)
            board.push(move)
            
            if is_white_move:
                chess_move = type('ChessMove', (), {
                    'move_number': move_number,
                    'white_move': san_move,
                    'black_move': None
                })()
                moves.append(chess_move)
            else:
                if moves and moves[-1].move_number == move_number:
                    moves[-1].black_move = san_move
                move_number += 1
        
        return metadata, moves
    
    def generate_game_summary(self, metadata: Dict, total_moves: int) -> str:
        """Generate final game summary"""
        result_text = {
            '1-0': f"{metadata['white']} wins",
            '0-1': f"{metadata['black']} wins", 
            '1/2-1/2': "The game ends in a draw"
        }.get(metadata['result'], "Game result unclear")
        
        summary = f"This was an excellent {total_moves}-move game between {metadata['white']} and {metadata['black']} "
        summary += f"from {metadata['event']}. {result_text}. "
        summary += f"Both players demonstrated solid opening preparation and tactical awareness. "
        summary += f"As always, feel free to leave any feedback in the comments section below. "
        summary += f"Hope you enjoyed it and maybe took a thing or two away. That's all for now. Take care!"
        
        return summary

# Initialize the real commentary generator
real_commentator = RealCommentaryGenerator()

# Main function to generate commentary
def generate_chess_commentary(pgn_input: str, style: str = "mixed") -> str:
    """
    Main function to generate chess commentary from PGN
    
    Args:
        pgn_input: PGN string of the chess game
        style: "mixed" (default), "jerry" (Chess Network focus), "hikaru" (Hikaru focus)
    
    Returns:
        Complete commentary as a formatted string
    """
    try:
        result = real_commentator.analyze_game_for_commentary(pgn_input)
        
        if 'error' in result:
            return f"Error: {result['error']}"
        
        # Format the complete commentary
        commentary = f"""
🎬 CHESS COMMENTARY - {result['metadata']['white']} vs {result['metadata']['black']}
{'='*80}

📋 GAME INFO:
   Event: {result['metadata']['event']}
   Date: {result['metadata']['date']}
   Result: {result['metadata']['result']}

🎙️ OPENING ANALYSIS:
{result['opening_commentary']}

⚔️ MIDDLEGAME BREAKDOWN:
{result['middlegame_commentary']}

🏁 ENDGAME EVALUATION:
{result['endgame_commentary']}

📝 FINAL THOUGHTS:
{result['full_game_summary']}

{'='*80}
        """
        
        return commentary.strip()
        
    except Exception as e:
        return f"Failed to generate commentary: {str(e)}"

print("✅ Chess Commentary Generator initialized successfully!")
print("Now run the next cell to see an example, or skip to the last cell to use your own PGN.")

✅ Chess Commentary Generator initialized successfully!
Now run the next cell to see an example, or skip to the last cell to use your own PGN.


In [2]:
# Now use YOUR PGN! Replace the example below with your chess game
your_pgn = """
[Event "Tata Steel Group A"]
[Site "Wijk aan Zee NED"]
[Date "2013.01.15"]
[EventDate "2013.01.12"]
[Round "4"]
[Result "0-1"]
[White "Levon Aronian"]
[Black "Viswanathan Anand"]
[ECO "D47"]
[WhiteElo "2802"]
[BlackElo "2772"]
[PlyCount "46"]

1. d4 d5 2. c4 c6 3. Nf3 Nf6 4. Nc3 e6 5. e3 Nbd7 6. Bd3 dxc4
7. Bxc4 b5 8. Bd3 Bd6 9. O-O O-O 10. Qc2 Bb7 11. a3 Rc8
12. Ng5 c5 13. Nxh7 Ng4 14. f4 cxd4 15. exd4 Bc5 16. Be2 Nde5
17. Bxg4 Bxd4+ 18. Kh1 Nxg4 19. Nxf8 f5 20. Ng6 Qf6 21. h3
Qxg6 22. Qe2 Qh5 23. Qd3 Be3 0-1
"""

print("🎬 Generating Commentary for Your Game...")
print("=" * 60)
commentary = generate_chess_commentary(your_pgn)
print(commentary)

print("\n💡 TIP: Replace the 'your_pgn' variable above with any PGN to get commentary!")

🎬 Generating Commentary for Your Game...
🎬 CHESS COMMENTARY - Levon Aronian vs Viswanathan Anand

📋 GAME INFO:
   Event: Tata Steel Group A
   Date: 2013.01.15
   Result: 0-1

🎙️ OPENING ANALYSIS:
Hi everyone, it's Jerry. I have an excellent game to share with you from Tata Steel Group A. This is a 2013.01.15 game between Levon Aronian playing white against Viswanathan Anand. We're looking at 1.d4 d5 2.c4 c6 3.Nf3 Nf6. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Levon Aronian could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME BREAKDOWN:
Now let's examine this position carefully. Levon Aronian has just played 7.Bxc4, and this creates several interesting possibilities. The key question here is how Levon Aronian should continue. If we look at 

In [3]:
# Example: Test with a famous game
example_pgn = """
[Event "World Championship Match"]
[Site "Reykjavik ISL"] 
[Date "1972.08.31"]
[Round "6"]
[White "Fischer, Robert J."]
[Black "Spassky, Boris V."]
[Result "1-0"]

1. c4 e6 2. Nf3 d5 3. d4 Nf6 4. Nc3 Be7 5. Bg5 O-O 6. e3 h6 7. Bh4 b6 
8. cxd5 Nxd5 9. Bxe7 Qxe7 10. Nxd5 exd5 11. Rc1 Be6 12. Qa4 c5 13. Qa3 Rc8 
14. Bb5 a6 15. dxc5 bxc5 16. O-O Ra7 17. Be2 Nd7 18. Nd4 Qf8 19. Nxe6 fxe6 
20. e4 d4 21. f4 Qe7 22. e5 Rb8 23. Bc4 Kh8 24. Qh3 Nf8 25. b3 a5 26. f5 exf5 
27. Rxf5 Nh7 28. Rcf1 Qd8 29. Qg3 Re7 30. h4 Rbb7 31. e6 Rbc7 32. Qe5 Qe8 
33. a4 Qd8 34. R1f2 Qe8 35. R2f3 Qd8 36. Bd3 Qe8 37. Qe4 Nf6 38. Rxf6 gxf6 
39. Rxf6 Kg8 40. Bc4 Kh8 41. Qf4 1-0
"""

print("🎯 Example: Fischer vs Spassky, 1972 World Championship")
print("=" * 60)
commentary = generate_chess_commentary(example_pgn)
print(commentary)

🎯 Example: Fischer vs Spassky, 1972 World Championship
🎬 CHESS COMMENTARY - Fischer, Robert J. vs Spassky, Boris V.

📋 GAME INFO:
   Event: World Championship Match
   Date: 1972.08.31
   Result: 1-0

🎙️ OPENING ANALYSIS:
Hi everyone, it's Jerry. I have an excellent game to share with you from World Championship Match. This is a 1972.08.31 game between Fischer, Robert J. playing white against Spassky, Boris V.. We're looking at 1.c4 e6 2.Nf3 d5 3.d4 Nf6. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Fischer, Robert J. could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME BREAKDOWN:
Now let's examine this position carefully. Fischer, Robert J. has just played 7.Bh4, and this creates several interesting possibilities. The key question here is how 

## Usage Instructions

### How to use this updated chess commentary generator:

1. **Simple Usage**: 
   ```python
   # Just paste your PGN and get commentary!
   your_pgn = "[Event...] 1. e4 e5 2. Nf3..."
   commentary = generate_chess_commentary(your_pgn)
   print(commentary)
   ```

2. **Advanced Usage**:
   ```python
   # Use the RealCommentaryGenerator class directly for more control
   generator = RealCommentaryGenerator()
   result = generator.analyze_game_for_commentary(your_pgn)
   ```

### What's different in this version:

✅ **Real Commentary Patterns** - Based on actual Chess Network and Hikaru transcripts  
✅ **Authentic Language** - Uses actual phrases and expressions from both creators  
✅ **Structured Output** - Generates opening, middlegame, endgame, and summary sections  
✅ **Easy Integration** - Simple function call generates complete commentary  
✅ **Mixed Styles** - Seamlessly blends Jerry's educational approach with Hikaru's entertainment  

### Key Features:

- **Chess Network Style**: "Hi everyone, it's Jerry", detailed analysis, computer evaluations
- **Hikaru Style**: "Chat, this is just winning", quick assessments, casual observations  
- **Game Phase Analysis**: Different commentary styles for opening, middlegame, endgame
- **Realistic Flow**: Natural transitions between analytical and entertaining commentary

In [5]:
# Test the new commentary generator with the Immortal Game
test_pgn = """
[Event "London"]
[Site "London"]
[Date "1851.06.21"]
[Round "?"]
[White "Anderssen, Adolf"]
[Black "Kieseritzky, Lionel"]
[Result "1-0"]

1. e4 e5 2. f4 exf4 3. Bc4 Qh4+ 4. Kf1 b5 5. Bxb5 Nf6 6. Nf3 Qh6 7. d3 Nh5 
8. Nh4 Qg5 9. Nf5 c6 10. g4 Nf6 11. Rg1 cxb5 12. h4 Qg6 13. h5 Qg5 14. Qf3 Ng8 
15. Bxf4 Qf6 16. Nc3 Bc5 17. Nd5 Qxb2 18. Bd6 Bxg1 19. e5 Qxa1+ 20. Ke2 Na6 
21. Nxg7+ Kd8 22. Qf6+ Nxf6 23. Be7# 1-0
"""

print("🎬 Generating Chess Network + Hikaru style commentary...")
print("=" * 60)

# Generate complete commentary
result = real_commentator.analyze_game_for_commentary(test_pgn)

if 'error' not in result:
    metadata = result['metadata']
    
    print(f"🎯 GAME INFO:")
    print(f"   White: {metadata['white']}")
    print(f"   Black: {metadata['black']}")
    print(f"   Event: {metadata['event']}")
    print(f"   Result: {metadata['result']}")
    
    print(f"\n🎙️ OPENING COMMENTARY:")
    print("-" * 40)
    print(result['opening_commentary'])
    
    print(f"\n⚔️ MIDDLEGAME COMMENTARY:")
    print("-" * 40)  
    print(result['middlegame_commentary'])
    
    print(f"\n🏁 ENDGAME COMMENTARY:")
    print("-" * 40)
    print(result['endgame_commentary'])
    
    print(f"\n📝 GAME SUMMARY:")
    print("-" * 40)
    print(result['full_game_summary'])
    
else:
    print(f"❌ Error: {result['error']}")

print("\n" + "=" * 60)

🎬 Generating Chess Network + Hikaru style commentary...
🎯 GAME INFO:
   White: Anderssen, Adolf
   Black: Kieseritzky, Lionel
   Event: London
   Result: 1-0

🎙️ OPENING COMMENTARY:
----------------------------------------
Hi everyone, it's Jerry. I have an excellent game to share with you from London. This is a 1851.06.21 game between Anderssen, Adolf playing white against Kieseritzky, Lionel. We're looking at 1.e4 e5 2.f4 exf4 3.Bc4 Qh4+. This demonstrates classical opening principles with both sides developing their pieces harmoniously. Now, going into this position, both players are following standard theory, but Anderssen, Adolf could have also considered other setups. Chat, this is pretty normal opening play. Like, both sides are just developing and fighting for the center. 

⚔️ MIDDLEGAME COMMENTARY:
----------------------------------------
Now let's examine this position carefully. Anderssen, Adolf has just played 7.d3, and this creates several interesting possibilities. The ke

## Example Usage and Demonstration

Below is a complete example showing how to use the chess commentary workflow with a sample game.

In [ ]:
@dataclass
class ChessMove:
    """Represents a chess move with analysis"""
    move_number: int
    white_move: Optional[str]
    black_move: Optional[str]
    position_before: str  # FEN
    position_after: str   # FEN
    evaluation: Optional[float] = None
    is_critical: bool = False
    move_type: str = "normal"  # "normal", "blunder", "brilliant", "sacrifice", etc.

class ChessGameParser:
    """Parses PGN format chess games"""
    
    def __init__(self):
        pass
    
    def parse_pgn(self, pgn_string: str) -> Tuple[Dict, List[ChessMove]]:
        """
        Parse a PGN string and return game metadata and moves
        """
        pgn_io = io.StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)
        
        if not game:
            raise ValueError("Invalid PGN format")
        
        # Extract metadata
        metadata = {
            'white': game.headers.get('White', 'Unknown'),
            'black': game.headers.get('Black', 'Unknown'),
            'result': game.headers.get('Result', '*'),
            'date': game.headers.get('Date', 'Unknown'),
            'event': game.headers.get('Event', 'Unknown'),
            'opening': game.headers.get('Opening', 'Unknown')
        }
        
        # Parse moves
        moves = []
        board = game.board()
        move_number = 1
        
        for i, move in enumerate(game.mainline_moves()):
            position_before = board.fen()
            
            # Determine if this is a white or black move
            is_white_move = board.turn == chess.WHITE
            
            # Make the move
            board.push(move)
            position_after = board.fen()
            
            # Create move object
            if is_white_move:
                # Starting a new move pair
                chess_move = ChessMove(
                    move_number=move_number,
                    white_move=board.san(move),
                    black_move=None,
                    position_before=position_before,
                    position_after=position_after
                )
                moves.append(chess_move)
            else:
                # Completing the move pair
                if moves and moves[-1].move_number == move_number:
                    moves[-1].black_move = board.san(move)
                    moves[-1].position_after = position_after
                else:
                    # Edge case: game starts with black move
                    chess_move = ChessMove(
                        move_number=move_number,
                        white_move=None,
                        black_move=board.san(move),
                        position_before=position_before,
                        position_after=position_after
                    )
                    moves.append(chess_move)
                move_number += 1
        
        return metadata, moves
    
    def analyze_move_criticality(self, moves: List[ChessMove]) -> List[ChessMove]:
        """
        Analyze moves to identify critical moments (simplified heuristic)
        """
        for i, move in enumerate(moves):
            # Simple heuristics for identifying critical moves
            # In a real implementation, you'd use a chess engine
            
            # Check for captures, checks, or piece sacrifices
            if move.white_move:
                if 'x' in move.white_move or '+' in move.white_move or '#' in move.white_move:
                    move.is_critical = True
                    if 'x' in move.white_move and any(piece in move.white_move for piece in ['Q', 'R', 'B', 'N']):
                        move.move_type = "capture"
                    elif '+' in move.white_move:
                        move.move_type = "check"
                    elif '#' in move.white_move:
                        move.move_type = "checkmate"
            
            if move.black_move:
                if 'x' in move.black_move or '+' in move.black_move or '#' in move.black_move:
                    move.is_critical = True
                    if 'x' in move.black_move and any(piece in move.black_move for piece in ['Q', 'R', 'B', 'N']):
                        move.move_type = "capture"
                    elif '+' in move.black_move:
                        move.move_type = "check"
                    elif '#' in move.black_move:
                        move.move_type = "checkmate"
        
        return moves

# Test the parser
parser = ChessGameParser()
print("Chess game parser initialized successfully!")